In [92]:
import os
import sys
sys.path.insert(0,'./tools/')

import numpy as np
import xarray as xr
import pandas as pd

import matplotlib.pyplot as plt

import cartopy.crs as ccrs

In [93]:
# Get properties and functions specific to S-MODE maps
from tools.config import MAPEXTENT
from maps import *

In [94]:
# Het ops area polygon and shore line from S-MODE Github
map_url = 'https://raw.githubusercontent.com/NASA-SMODE/Maps/main/tools/' 
shore = pd.read_json(map_url + 'NorthCalShoreLine.json')
opsarea = pd.read_json(map_url + 'ops_area_polygon.json')

In [95]:
# Figure settings
figdir  = 'img/'
figname = 'SSTSnapshot'
extension = ['png']
figproperties = dict(dpi=200,bbox_inches='tight')

### Fetching some data from WHOI's S-MODE thredds

In [ ]:
# The 10/14 VIIRS SST image looks particularly good
base_url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/'
filename = 'VIIRS_NRT/VIIRS_NRT_20211014T102000Z.nc'

# Put the data into a Dataset and select data within MAPEXTENT
sst = xr.open_dataset(base_url+filename).isel(time=0)
sst = sst.where((sst.lon>=MAPEXTENT[0])&
                (sst.lon<=MAPEXTENT[1])&
                (sst.lat>=MAPEXTENT[2])&
                (sst.lat<=MAPEXTENT[3]),
                 drop=True)

In [ ]:
sst

In [ ]:
# AVISO's SLA may also be useful
base_url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/'
aviso = xr.open_dataset(base_url+'AVISO/aviso.nc').sel(time='2021-10-14T00:00:00.000000000')

# AVISO data is on a regular grid, with longitude and latitude as coordinates, 
# so we can slice the dataset easily
d = 0.25
aviso = aviso.sel(longitude=slice(MAPEXTENT[0]-d,MAPEXTENT[1]+d),
                  latitude=slice(MAPEXTENT[2]-d,MAPEXTENT[3]+d)
)

In [ ]:
aviso

# A quick-and-dirty plot

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)

# Plot shoreline and operations area
shore.plot(x = 'longitude',
           y = 'latitude', 
           color = 'k',
           legend=False,
           ax = ax
)

opsarea.plot(x = 'longitude',
             y = 'latitude', 
             color = 'k',
             legend=False,
             ax = ax
)

# plot SST image
kwargs_t = {
    'vmin': 11.5,
    'vmax': 15.5,
    'cmap': 'Spectral_r',
    'add_colorbar': False
}

im = (sst.sea_surface_temperature-273.15).\
                plot.pcolormesh(x='lon',
                                y='lat',
                                **kwargs_t
                )

# contour absolute dynamic topography
kwargs_a = {
    'levels': np.arange(-0.2,0.201,.01),
    'colors': '0.25',
    'linewidths': 0.75
}

(aviso.adt-aviso.adt.mean()).plot.contour(**kwargs_a,ax=ax)

ax.set_xlim(*MAPEXTENT[:2])
ax.set_ylim(*MAPEXTENT[2:])

plt.colorbar(im,label=r'SST [$^\circ$C]',
             shrink=0.75,extend='both')

## A map with projection using cartopy

In [ ]:
projection = ccrs.PlateCarree(
                central_longitude=(MAPEXTENT[0]+MAPEXTENT[1])/2
)

map_axes = (GeoAxes,{'map_projection':projection})
trans = ccrs.PlateCarree()

In [ ]:
fig = plt.figure(figsize=(8,7))

ax = fig.add_subplot(111, projection=projection)     

ax.set_extent(MAPEXTENT)


# Plot shoreline and operations area
opsarea.plot(x = 'longitude',
             y = 'latitude', 
             color = 'k',
             legend=False,
             ax = ax,
             transform=ccrs.PlateCarree()
)

# plot sst image
im = (sst.sea_surface_temperature-273.15).\
                plot.pcolormesh(x='lon',
                                y='lat',
                                transform=trans,
                                **kwargs_t)

# contour absolute dynamic topography
(aviso.adt-aviso.adt.mean()).plot.contour(ax=ax, 
                                          transform=trans,
                                           **kwargs_a)


# map properties (continent, labels, grid)
plot_map_properties(ax,
                    transform=ccrs.PlateCarree(),
                    continent_facecolor='0.35')

fig.colorbar(im,shrink=0.5,extend='both',
             label=r'SST [C$^\circ$]')

# Save figure in the desired extensions
[fig.savefig(os.path.join(figdir,figname+'_cartopy.'+ext),
             **figproperties) 
             for ext in extension]